In [ ]:
###Models to do
# models on cleaned data
# models on averages/trend data
# models on new data (eg. PAV, ladder, ohe venue)
    #make sure to include absolute margin when testing and evaluating these
    #keep track in excel?
# lazy predict

In [1]:
import re
import xgboost as xgb
import tensorflow as tf; print(tf.__version__)
import keras; print(keras.__version__)
#import torch.nn as nn
#import touch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from random import randint
from Gather_AFL_Data import gatherer as gad
#from fdnn import feature_extractor as fex
import skopt
from skopt.searchcv import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import StratifiedKFold
import pickle
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Activation
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import cross_val_score, KFold

2.2.0
2.4.3


In [2]:
#get headers
#feed this into a bigger function which specifies the amount of games to go through 
def get_headers(n_games):
    headers = ['Round', 'Home_Team', 'Away_Team']
    example_file = pd.read_csv('Data/Fremantle_clean_stats.csv')
    cl_h = example_file.columns
    cl_h = cl_h[:-5]
    ladder_header = ['Ladder Pos_H', 'Form_H', 'Season Wins_H', 'Season Loss_H', 'Season Draw_H']
    headers = [*headers, *ladder_header]
    j = 1
    while j <= n_games:
        for x in cl_h:
            if 'Match_ID' in x or 'Year' in x:
                continue
            x = 'H_'+ x + ' n-' + str(j)
            headers.append(x)
        j = j + 1
    j = 1
    ladder_header = ['Ladder Pos_A', 'Form_A', 'Season Wins_A', 'Season Loss_A', 'Season Draw_A']
    headers = [*headers, *ladder_header]
    while j <= n_games:
        for x in cl_h:
            if 'Match_ID' in x or 'Year' in x:
                continue
            x = 'A_'+ x + ' n-' + str(j)
            headers.append(x)
        j = j + 1
    return headers


In [3]:
def clean_headers(h):
    headers = []
    for x in h:
        if '<' in x or '>' in x:
            x = x.replace('<',"lt_")
            x = x.replace('>', "gt_")
            #print(x)
        headers.append(str(x))
    return headers

In [4]:
def generate_categorical_headers(h):
    cat_var = ['Round', 'Home_Team', 'Away_Team']
    skip = 0
    for x in h:
        if 'Round' in x:
            if (skip == 0):
                skip = 1
                continue
            cat_var.append(x)
            #print(x)
        elif 'Team_against_ID' in x:
            #print(x)
            cat_var.append(x)
        elif 'Venue' in x:
            cat_var.append(x)
    return cat_var

In [5]:
#one hot encode data and transform the X_data
#first redo, find the categorial variables
def ohe_data(x_data, enc, flag,cat_var):
    #data has not been previously one hot encoded
    if (flag == 0):
        #get columns with categorical data and drop from main DF
        categorical_data = x_data[cat_var]
        x_data = x_data.drop(cat_var, axis = 1)
        #define and fit new OHE. Use it on our categorical data by transforming
        ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
        ohe = ohe.fit(categorical_data)
        categorical_data = ohe.transform(categorical_data)
        #get feature names for better labelling
        fn = ohe.get_feature_names(cat_var)
        #make a dataframe with new OHE data and feature names
            #would have been good to have coded it like this for my Masters project...
        categorical_data = pd.DataFrame(categorical_data)
        categorical_data.columns = fn
        #ensure that it won't get cranky about any different indexes(shouldn't be any but just a good check)
        x_data.reset_index(drop=True, inplace=True)
        categorical_data.reset_index(drop=True, inplace=True)
        #concatenate along column axis
        x_data = pd.concat([x_data, categorical_data], axis = 1)
    else:
        #same as above except used already fitted ohe
        categorical_data = x_data[cat_var]
        x_data = x_data.drop(cat_var, axis = 1)
        categorical_data = enc.transform(categorical_data)
        fn = enc.get_feature_names(cat_var)
        categorical_data = pd.DataFrame(categorical_data)
        categorical_data.columns = fn
        x_data.reset_index(drop=True, inplace=True)
        categorical_data.reset_index(drop=True, inplace=True)
        x_data = pd.concat([x_data, categorical_data], axis = 1, ignore_index=True)
        ohe = enc
    return x_data, ohe

In [17]:
#search parameters for best XGB classifier or best XGB regressor

def param_search(x_data, y_label, class_reg):

    def on_step(optim_result):
        """
        Callback meant to view scores after
        each iteration while performing Bayesian
        Optimization in Skopt"""
        score = xgb_bayes_search.best_score_
        print("best score: %s" % score)
        if score >= 0.98:
            print('Interrupting!')
            return True
    X_train, X_test, y_train, y_test = train_test_split(x_data, y_label, test_size=0.1, random_state=157732)
    print("X_train shape: " + str(X_train.shape))
    print("X_test shape: " + str(X_test.shape))
    print("y_train shape: " + str(y_train.shape))
    print("y_test shape: " + str(y_test.shape))
    space ={'learning_rate': Real(0.01, 1.0, 'log-uniform'),
        'min_child_weight': Integer(0, 10),
        'max_depth': Integer(0, 50),
        'max_delta_step': Integer(0, 20),
        'subsample': Real(0.01, 1.0, 'uniform'),
        'colsample_bytree': Real(0.01, 1.0, 'uniform'),
        'colsample_bylevel': Real(0.01, 1.0, 'uniform'),
        'reg_lambda': Real(1e-9, 1000, 'log-uniform'),
        'reg_alpha': Real(1e-9, 1.0, 'log-uniform'),
        'gamma': Real(1e-9, 0.5, 'log-uniform'),
        'min_child_weight': Integer(0, 5),
        'n_estimators': Integer(50, 200),
        'scale_pos_weight': Real(1e-6, 500, 'log-uniform')}
    if(class_reg == 0):
        xgbclass = xgb.XGBClassifier(random_state=27022013)
    else:
        xgbclass = xgb.XGBRegressor(random_state=27022013)
    xgb_bayes_search = BayesSearchCV(xgbclass, space, n_iter=60, # specify how many iterations
                                    scoring=None, n_jobs=1, cv=10, verbose=1, random_state=42, n_points=12,
                                 refit=True)
    kk = np.isinf(X_train)
    if True in kk:
    	print("aaaaaaa")
    kk = np.isinf(y_train)
    if True in kk:
    	print("reeeeeee")
    try:
        xgb_bayes_search.fit(X_train, y_train, callback = on_step)
    except:
        xgb_bayes_search.fit(X_train, y_train, callback = on_step)
    print("BEST PARAMS ARE HERE")
    print(xgb_bayes_search.best_params_)
    model = xgb_bayes_search.best_estimator_

    return model

ERROR! Session/line number was not unique in database. History logging moved to new session 40


In [15]:
#x = all data
#y = label of whether home team or away team won
#m = xgb classifier for winner prediction
#my = labels of margins
    #try absolute values
#mm = xgb regressor for margin prediction
def eval_xgb_games_margins(x, y, m, my, mm):
    results = []
    error = []
    count = 0
    best_w = m
    high_w = 0
    best_m = mm
    high_m = 100
    cv = StratifiedKFold(n_splits=10, shuffle=True)
    print(x)
    for train,test in cv.split(x,y):
       # print(len(train))
        count = count + 1
        print("Split: " + str(count))
        #comment out fit steps for random forest i guess lol
        prediction = m.fit(x.loc[train],y.loc[train]).predict_proba(x.loc[test])
        margin_pred = mm.fit(x.loc[train], my.loc[train])
        print("variables for auroc curve done. Processing fold accuracy + checking best model")
        y_pred = m.predict(x.loc[test])
        #print(y_pred)
        m_pred = mm.predict(x.loc[test])
        #print(m_pred)
        predictions = [round(value) for value in y_pred]
        #sees how accurate the model was when testing the test set
        accuracy = accuracy_score(y.loc[test], predictions)
        pcent = accuracy * 100.0
       # print("The accuracy of this model is" + str(pcent))
        rmse = sqrt(mean_squared_error(m_pred, my.loc[test]))
       # print("The rmse of this model is" + str(rmse))
        results.append(pcent)
        error.append(rmse)
        #change the best model to equal current model
        if(pcent > high_w):
            print("found new best classify")
            best_w = m
            high_w = pcent
        if(rmse < high_m):
            print("found best new margin")
            best_m = mm
            high_m = rmse
    print("Best win percentage split = " +str(high_w))
    print("Best margin rmse = "+str(high_m))
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    print("Training Testing Margins: %.2f%% (%.2f%%)" % (np.mean(error), np.std(error)))
    return best_w, best_m

In [16]:
#make this a method param later
#n_games = 5
def run_all_models(n_games):
    #load headers and then subsequently categorical headers
    h = get_headers(n_games)
    h = clean_headers(h)
    cv = generate_categorical_headers(h)

    #to get names of teams from index
    g = gad()
    teams = g.createTeamDict()

    #load Data
    x_data = pd.read_csv('Data/assembled_stat_matrix_no2020'+str(n_games)+'_games.csv')

    #make empty OHE object
    na_enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

    #one hot encode data with new one hot encoder, saves ohe for later use
    x_data, ohe = ohe_data(x_data, na_enc, 0, cv)

    #reset headers
    feature_names = x_data.columns

    #loads the ylabel matrix,
    y_label = pd.read_csv('Data/assembled_labelled_ymatrix_no2020'+str(n_games)+'_games.csv')

    #loads margin as the y_label
    margin_label = pd.read_csv('Data/assembled_margin_ymatrix_no2020'+str(n_games)+'_games.csv')

    print(margin_label.shape)
    print(y_label.shape)
    print(x_data.shape)

    #regex solution which is apparently necessary??
    regex = re.compile(r"\[|\]|<", re.IGNORECASE)
    x_data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in x_data.columns.values]

    #optimise XGBoost model

    #for predicting win
    model = param_search(x_data, y_label, 0)
    #for predicting margin
    margin_label = abs(margin_label)
    margin_model = param_search(x_data, margin_label, 1)

    margin_label = abs(margin_label)

    best_xgb_clas, best_xgb_reg = eval_xgb_games_margins(x_data, y_label, model, margin_label, margin_model)

    #save
    best_xgb_clas.save_model("Models/best_xgb_clas_no2020_"+str(n_games)+'_games.bin')
    best_xgb_reg.save_model("Models/best_xgb_reg_no2020_"+str(n_games)+'_games.bin')

In [18]:
#as one big script to go through n_games
n_games = [1,2,3,4,5,6,7,10]
for n in n_games:
    run_all_models(n)

(1646, 1)
(1646, 1)
(1646, 387)
X_train shape: (1481, 387)
X_test shape: (165, 387)
y_train shape: (1481, 1)
y_test shape: (165, 1)
Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:   31.6s finished


best score: 0.636731937879811
Fitting 10 folds for each of 12 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:  1.4min finished


best score: 0.636731937879811
Fitting 10 folds for each of 12 candidates, totalling 120 fits


...

variables for auroc curve done. Processing fold accuracy + checking best model
Split: 9


/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


variables for auroc curve done. Processing fold accuracy + checking best model
Split: 10


/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


variables for auroc curve done. Processing fold accuracy + checking best model
Best win percentage split = 71.3375796178344
Best margin rmse = 23.540348381063346
Training Testing Accuracy: 65.11% (3.42%)
Training Testing Margins: 26.05% (1.36%)


# Other Shit Model Testing

In [ ]:
#lazy predict

In [27]:
from sklearn.ensemble import *

#random forest models
#rf_model = RandomForestClassifier(max_depth=2, random_state=0)
rf_margin_model = RandomForestRegressor(max_depth=2, random_state=0)

#SVM
from sklearn.svm import SVC
from sklearn.svm import SVR
svm_model = SVC(gamma='auto', probability=True)
svm_margin_model = SVR(kernel = 'rbf')

#BNB
from sklearn.naive_bayes import BernoulliNB
bnb_model = BernoulliNB()

In [23]:
from sklearn.model_selection import RandomizedSearchCV
#optimise rf models
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_model = RandomizedSearchCV(estimator = rf_model, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1)


#TRAIN TEST SPLIT HERE 80:20

# Fit the random search model
#CHANGE TO X_TRAIN etc.
rf_model.fit(x_data, y_label)

#do the usual test eval here


rf_margin_model = RandomizedSearchCV(estimator = rf_margin_model, param_distributions = random_grid, n_iter = 100, cv = 10, verbose=2, random_state=42, n_jobs = -1)

# Fit the random search model
rf_margin_model.fit(x_data, margin_label)

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed: 20.9min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed: 50.4min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 81.5min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 124.1min finished


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  3.0min
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/chris/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-23-192914c5fafe>", line 31, in <module>
    rf_margin_model.fit(x_data, margin_label)
  File "/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py", line 710, in fit
    self._run_search(evaluate_candidates)
  File "/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py", line 1482, in _run_search
    evaluate_candidates(ParameterSampler(
  File "/home/chris/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_search.py", line 682, in evaluate_candidates
    out = parallel(delayed(_fit_and_score)(clone(base_estimator),
  File "/home/chris/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 1054, in __call__
    self.retrieve()
  File "/home/chris/anaconda3/lib/python3.8/site-pack

TypeError: object of type 'NoneType' has no len()

In [ ]:
def build_DNN_model(x_len):
    model = Sequential()
    model.add(Dense(63, input_dim = x_len))
    model.add(Activation('relu'))
    model.add(Dropout(0.03))
    model.add(BatchNormalization())

    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.02))

    model.add(Dense(16))
    model.add(Activation('relu'))

    model.add(Dense(8))
    model.add(Activation('relu'))
    #add output layer
    model.add(Dense(1, activation='linear'))
    opt = tf.keras.optimizers.Adamax(learning_rate=0.003)

    model.compile(loss="mean_squared_error", optimizer=opt, metrics=[tf.keras.metrics.RootMeanSquaredError()])
    print(model.summary())
    return model

In [ ]:
def build_CNN_model(x_len):
    #del model
    model = Sequential()
    model.add(Conv1D(filters=32, kernel_size=14,
                     input_shape=(x_len, 1)))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Conv1D(filters=16, kernel_size=10,
                     input_shape=(32, 1)))
    model.add(Activation('linear'))
    model.add(Dropout(0.1))
    model.add(Conv1D(filters=10, kernel_size=8,
                     input_shape=(16, 1)))
    model.add(Activation('linear'))
    model.add(MaxPooling1D(pool_size=2))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(64, activation='linear'))
    model.add(Dense(32, activation='linear'))
    model.add(Dense(16, activation='linear'))
    model.add(BatchNormalization())
    model.add(Dense(1, activation='linear'))
    opt = tf.keras.optimizers.Adamax(learning_rate=0.003)#, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name="Adamax"


    model.compile(loss="mean_squared_error", optimizer=opt, metrics=[tf.keras.metrics.RootMeanSquaredError()])

    print(model.summary())
    return model

In [ ]:
#flag = 0 (DNN)
#flag = 1 (CNN)
def eval_dl(x,y,k,flag):
    cv = StratifiedKFold(n_splits=k,shuffle=True)
    best_model = []
    results = []
    highest = 0
    i = 1
    for train,test in cv.split(x,y):
        if(flag == 0):
            model = build_DNN_model(x[train].shape[1])
        if(flag == 1):
            x = x.reshape(x.shape[0], x.shape[1], 1)
            model = build_CNN_model(x[train].shape[1])
        bs = ((x[train].shape[0])/20)
        bs = round(bs)
        history = model.fit(x[train], y[train], validation_data=(x[test], y[test]), epochs = 50, batch_size=bs)
        _, accuracy = model.evaluate(x[test], y[test], batch_size=bs, verbose=0)
        accuracy = accuracy * 100
        print("accuracy for model " + str(i) + " is " + str(accuracy))
        if(accuracy > highest):
            highest = accuracy
            best_model = model
        results.append(accuracy)
        i = i + 1
    print("highest accuracy is: " + str(highest))
    print("Training Testing Accuracy: %.2f%% (%.2f%%)" % (np.mean(results), np.std(results)))
    return best_model